<a href="https://colab.research.google.com/github/serlysetyani/Forensic-Digital_Duplication-Region/blob/main/Forensic_SIFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
# Library
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
img = cv2.imread("photo-1559736220-66fc1882555d-F2.jpg")
h, w, c = img.shape
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
sift = cv2.SIFT_create(nfeatures=100000, nOctaveLayers=7, sigma=1.6)
keypoints, descriptors = sift.detectAndCompute(img_gray, None)

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches = bf.knnMatch(descriptors, descriptors, k=3)

better_matches = []
for a, b, c in matches:
    if a.trainIdx == a.queryIdx:
        better_matches.append([b, c])
    elif b.trainIdx == b.queryIdx:
        better_matches.append([a, c])
    elif c.trainIdx == c.queryIdx:
        better_matches.append([a, b])

ratio_thresh = 0.5
good_matches = []
for m, n in better_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)


MIN_MATCH_COUNT = 3
if len(good_matches) > MIN_MATCH_COUNT:
    src_pts = np.float32([keypoints[m.trainIdx].pt for m in good_matches])
    dst_pts = np.float32([keypoints[m.queryIdx].pt for m in good_matches])

    retval, inliers = cv2.estimateAffine2D(src_pts, dst_pts, method=cv2.RANSAC, ransacReprojThreshold=3, maxIters=100,
                                           confidence=0.99)

    matchesMask = inliers.ravel().tolist()

In [ ]:
final_matches = []
for i in range(len(good_matches)):
    if matchesMask[i] == 1:
        final_matches.append(good_matches[i])

img_RGB = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)

list_point1 = []
list_point2 = []
for j in final_matches:

    # Get the matching keypoints for each of the images
    point1 = j.trainIdx
    point2 = j.queryIdx

    # Get the coordinates, x - columns, y - rows
    (x1, y1) = keypoints[point1].pt
    (x2, y2) = keypoints[point2].pt

    # Append to each list
    list_point1.append((int(x1), int(y1)))
    list_point2.append((int(x2), int(y2)))

    # Draw a small circle at both co-ordinates: radius 4, colour green, thickness = 1
    # copy keypoints circles
    cv2.circle(img_RGB, (int(x1), int(y1)), 4, (0, 255, 0), 1)
    # original keypoints circles
    cv2.circle(img_RGB, (int(x2), int(y2)), 4, (0, 255, 0), 1)

    # Draw a line in between the two points, thickness = 1, colour green
    cv2.line(img_RGB, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1)

In [ ]:
wrapAffine_img = cv2.warpAffine(img_gray, retval, (w, h))
cv2.imwrite('res.png', wrapAffine_img)

blank_image = np.zeros((h, w, 1), np.uint8)

for y in range(0, h-3):
    for x in range(0, w-3):
        window1 = img_gray[y: y+3, x: x+3]
        window2 = wrapAffine_img[y: y + 3, x: x + 3]
        a1 = window1[1][1]
        a2 = window2[1][1]
        mean1 = cv2.mean(window1)
        mean2 = cv2.mean(window2)
        mean1_num = mean1[0]
        mean2_num = mean2[0]
        b1 = a1 - mean1_num
        b2 = a2 - mean2_num
        top = (b1*b2)
        bottom = math.sqrt((b1*b1)*(b2*b2))

        if bottom > 0:
            print(top/bottom)
            intensity = top/bottom
            blank_image[y + 1, x + 1] = intensity
        elif bottom <= 0:
            intensity = 0
            blank_image[y+1, x+1] = intensity

cv2.imwrite("blank_image.png", blank_image)

In [ ]:
cv2.imshow("result", res)
cv2.imwrite('final_matches.png', res)
#cv2.imshow("corelation", img_correlation_map)
cv2.waitKey(0)
cv2.destroyAllWindows()